In [1]:
import pandas as pd
import re
import nltk
import os
from collections import Counter
from nltk.util import ngrams
from nltk.corpus import stopwords

# NLTK stopword listesini indir (eğer daha önce indirilmemişse)
#nltk.download('stopwords')

# Set the path as tangible
os.chdir('..')

# Check current path 
print(os.getcwd())


c:\Users\user\Documents\Python Projects\tangible


In [11]:
# Read the text file
file_path = 'data/ankara_oe/article_1977_1.txt'
base_name = os.path.basename(file_path)
document_name = os.path.splitext(base_name)[0]
print("Document: " + document_name)

# Metin dosyası
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

#stop_words = set(stopwords.words('turkish'))
custom_stopwords = set([
    "ama", "bir", "bu", "çok", "daha", "gibi", "için", "ile", "ise", "önemli",  "şu", "ve"
])

# Metni küçük harfe çevirme ve temizleme
text = text.lower()
text = re.sub(r'[^\w\s]', '', text)  # Noktalama işaretlerini kaldır

# Kelimeleri tokenize et ve stopword’leri kaldır
tokens = [word for word in text.split() if word not in custom_stopwords]

# Unigram, bigram ve trigramları oluştur
unigrams = tokens
bigrams = list(ngrams(tokens, 2))
trigrams = list(ngrams(tokens, 3))
quadrigram = list(ngrams(tokens, 4))

# Frekansları hesapla
unigram_freq = Counter(unigrams)
bigram_freq = Counter(bigrams)
trigram_freq = Counter(trigrams)
quadrigram_freq = Counter(quadrigram)

# En sık geçenleri seç
#top_unigrams = unigram_freq.most_common(10)
top_bigrams = bigram_freq.most_common(10)
top_trigrams = trigram_freq.most_common(10)
top_quadrigrams = quadrigram_freq.most_common(10)

# Sonuçları DataFrame olarak hazırlama
df_unigram = pd.DataFrame(unigram_freq.items(), columns=["Unigram", "Frekans"]).sort_values(by="Frekans", ascending=False)

df_bigram = pd.DataFrame([" ".join(bigram) for bigram, freq in top_bigrams], columns=["Bigram"])
df_bigram["Frekans"] = [freq for bigram, freq in top_bigrams]

df_trigram = pd.DataFrame([" ".join(trigram) for trigram, freq in top_trigrams], columns=["Trigram"])
df_trigram["Frekans"] = [freq for trigram, freq in top_trigrams]

df_quadrigram = pd.DataFrame(quadrigram_freq.items(), columns=["Quadrigram", "Frekans"]).sort_values(by="Frekans", ascending=False)


# Unigramları Görüntüle
#print("\n🔹 Unigram Frekansları:")
#print(df_unigram)

# Bigramları Görüntüle
#print("\n🔹 Bigram Frekansları:")
#print(df_bigram)

# **Trigramları Görüntüle**
#print("\n🔹 Trigram Frekansları:")
#print(df_trigram)

# Quadrigramları Görüntüle
print("\n🔹 Quadrigram Frekansları:")
print(df_quadrigram)


Document: article_1977_1

🔹 Quadrigram Frekansları:
                                    Quadrigram  Frekans
1256         (özel, eğitime, muhtaç, çocuklar)        5
4350  (kendi, şartlarımıza, göre, hazırlanmış)        5
4354   (geçerli, güvenilir, zekâ, testlerinde)        5
4355   (güvenilir, zekâ, testlerinde, devamlı)        5
4356      (zekâ, testlerinde, devamlı, olarak)        5
...                                        ...      ...
7959            (dil, kurumu, yayınları, özel)        1
7960         (kurumu, yayınları, özel, eğitim)        1
7961         (yayınları, özel, eğitim, bölümü)        1
7962           (özel, eğitim, bölümü, öğretim)        1
9       (terimlerin, çokluğu, türkiyede, özel)        1

[7964 rows x 2 columns]


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=list(stop_words))
tfidf_matrix = vectorizer.fit_transform([text])

# Kelimelerin önem sırasına göre alınması
feature_names = vectorizer.get_feature_names_out()
scores = tfidf_matrix.toarray().flatten()

# En önemli kelimeleri sıralama
important_terms = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)

# Tablo olarak gösterme
df_tfidf = pd.DataFrame(important_terms, columns=["Terim", "TF-IDF Skoru"])
df_tfidf = df_tfidf.sort_values(by="TF-IDF Skoru", ascending=False)
print(df_tfidf.shape)
print(df_tfidf)


(24786, 2)
                                  Terim  TF-IDF Skoru
0                                   bir      0.753827
1                                 kadar      0.100176
2                                 sonra      0.095168
3                                   ben      0.082646
4                                eğitim      0.082646
...                                 ...           ...
24749                           şişenin      0.002504
24750               şişenin serinliğini      0.002504
24751  şişenin serinliğini hissediyorum      0.002504
24752                            şişeyi      0.002504
21687                       yazabiliyor      0.002504

[24786 rows x 2 columns]


In [ ]:
# Anahtar Sözcükler
keywords = [
    "az",
    "bütünleştirme"
    "görme",
    "kaynaştırma",
    "otizm",
    "özel",
    "yetersizli",
    "zihin"
]

# **Önemli terimleri filtreleme (TF-IDF sonucundan)**
filtered_terms = [term for term, score in important_terms if any(keyword in term for keyword in keywords)]

# **Sonucu DataFrame olarak saklama**
df_filtered = pd.DataFrame(filtered_terms, columns=["Özel Eğitim Terimleri"])

# **Frekansları hesaplama (unigram, bigram, trigram içinde arama)**
filtered_frequencies = {}

for term in df_filtered["Özel Eğitim Terimleri"]:
    # Unigram frekansı
    unigram_count = unigram_freq.get(term, 0)
    # Bigram frekansı
    bigram_count = bigram_freq.get(tuple(term.split()), 0)
    # Trigram frekansı
    trigram_count = trigram_freq.get(tuple(term.split()), 0)

    # Toplam frekans
    total_count = unigram_count + bigram_count + trigram_count
    filtered_frequencies[term] = total_count

# **Yeni DataFrame oluştur (terimler ve frekansları)**
df_final = pd.DataFrame(filtered_frequencies.items(), columns=["Terim", "Frekans"])

# **Sonucu gösterme**
print("\n🔹 Filtrelenmiş Terimlerin Frekansları:")
print(df_final)



🔹 Filtrelenmiş Terimlerin Frekansları:
                         Terim  Frekans
0                         özel       29
1                  özel eğitim       20
2            özel eğitim çocuk       16
3                        bazen       15
4                        biraz        6
...                        ...      ...
1160  şiirler yazmaya başladım        1
1161                 şti kazım        1
1162       şti kazım karabekir        1
1163              şunları özel        1
1164       şunları özel eğitim        1

[1165 rows x 2 columns]


In [21]:
# Analiz edilecek özel eğitim terimleri
special_ed_terms = [
    "az gören",
    "bireysel", "bireysel farklılık", "bireysel farklılıklar", "bireyselleştirme", "bireyselleştirilmiş",
    "bireyselleştirilmiş eğitim", "bireysel eğitim",
    "bireyselleştirilmiş eğitim programı", "bireyselleştirilmiş eğitim planı",
    "destek eğitim", "destek eğitimi",
    "görme engeli", "görme engelli", "görme yetersizliği", "görme yetersizliği olan",
    "kör",
    "otizm", "otistik", "otizm spektrum bozukluğu",
    "özel eğitim",
    "özel eğitim hizmeti", "özel eğitim hizmetleri",
    "özel gereksinim", "özel gereksinimli", "özel eğitime ihtiyacı olan", "özel eğitime muhtaç",
    "rehabilitasyon",
    "yetersizlik", "yetersizliği olan",
    "zeka geriliği", "geri zekalı",
    "zihin yetersizliği", "zihinsel yetersizlik", "zihinsel yetersizliği olan"
]

# Terimlerin frekanslarını hesaplama
term_frequencies = {}

for term in special_ed_terms:
    term_tuple = tuple(term.split())  # Terimi tuple formatına çevir
    
    # 1 Öncelikle quadrigram içinde olup olmadığını kontrol et
    quadrigram_count = quadrigram_freq.get(term_tuple, 0)
    
    # 2 Eğer quadrigram içinde varsa diğerlerini yok say
    if quadrigram_count > 0:
        term_frequencies[term] = quadrigram_count
        continue

    # 3 Eğer quadrigram yoksa, trigram'ları kontrol et
    trigram_count = trigram_freq.get(term_tuple, 0)

    if trigram_count > 0:
        term_frequencies[term] = trigram_count
        continue

    # 4 Eğer trigram yoksa, bigram'ları kontrol et
    bigram_count = bigram_freq.get(term_tuple, 0)

    if bigram_count > 0:
        term_frequencies[term] = bigram_count
        continue

    # 5 Diğerlerinde yoksa unigram'ı kontrol et
    unigram_count = unigram_freq.get(term, 0)
    
    # 6 Sonuçları kaydet
    term_frequencies[term] = unigram_count

# **Sonuçları DataFrame olarak kaydet**
df_special_terms = pd.DataFrame(term_frequencies.items(), columns=["Terim", "Frekans"]).sort_values(by="Frekans", ascending=False)
df_special_terms['Source'] = document_name

# **Sonuçları gösterme**
print("\n🔹 Özel Eğitim Terimlerinin Frekansları:")
print(df_special_terms[df_special_terms["Frekans"] > 0])



🔹 Özel Eğitim Terimlerinin Frekansları:
                         Terim  Frekans          Source
20                 özel eğitim       41  article_1977_1
16                         kör       25  article_1977_1
26         özel eğitime muhtaç        7  article_1977_1
27              rehabilitasyon        4  article_1977_1
28                 yetersizlik        4  article_1977_1
17                       otizm        4  article_1977_1
0                     az gören        2  article_1977_1
22      özel eğitim hizmetleri        2  article_1977_1
25  özel eğitime ihtiyacı olan        2  article_1977_1
29           yetersizliği olan        1  article_1977_1
1                     bireysel        1  article_1977_1
